Etape 1: Creation du DataSet

In [1]:
import pandas 
import numpy as np
import matplotlib.pyplot as plt


df_3features=pandas.read_csv("./datasets/Stress_temperature_step_humidity.csv", sep =',')
df_health=pandas.read_csv("./datasets/Sleep_health.csv", sep =',')
df_sleep=pandas.read_csv("./datasets/sayopillow/SaYoPillow.csv", sep =',')

df_health.drop(["Occupation","Blood Pressure"], axis = 1, inplace=True)





In [2]:
df_health.drop( labels="Person ID", axis = 1, inplace= True)
df_health.drop( labels="BMI Category", axis = 1, inplace = True)

In [3]:
df_health.iloc[:20]

,Gender,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Sleep Disorder
0,Male,27,6.1,6,42,6,77,4200,NaN
1,Male,28,6.2,6,60,8,75,10000,NaN
2,Male,28,6.2,6,60,8,75,10000,NaN
3,Male,28,5.9,4,30,8,85,3000,Sleep Apnea
4,Male,28,5.9,4,30,8,85,3000,Sleep Apnea
5,Male,28,5.9,4,30,8,85,3000,Insomnia
6,Male,29,6.3,6,40,7,82,3500,Insomnia
7,Male,29,7.8,7,75,6,70,8000,NaN
8,Male,29,7.8,7,75,6,70,8000,NaN
9,Male,29,7.8,7,75,6,70,8000,NaN


In [5]:

#6 features 
print(df_health["Sleep Disorder"].unique())
print(df_health["Sleep Duration"].unique())
print(df_health["Quality of Sleep"].unique())
print(df_health["Stress Level"].unique())
print(df_health["Physical Activity Level"].unique())
print(df_health["Heart Rate"].unique())
print(df_health["Daily Steps"].unique())

[nan 'Sleep Apnea' 'Insomnia']
[6.1 6.2 5.9 6.3 7.8 6.  6.5 7.6 7.7 7.9 6.4 7.5 7.2 5.8 6.7 7.3 7.4 7.1
 6.6 6.9 8.  6.8 8.1 8.3 8.5 8.4 8.2]
[6 4 7 5 8 9]
[6 8 7 4 3 5]
[42 60 30 40 75 35 45 50 32 70 80 55 90 47 65 85]
[77 75 85 82 70 80 78 69 72 68 76 81 65 84 74 67 73 83 86]
[ 4200 10000  3000  3500  8000  4000  4100  6800  5000  7000  5500  5200
  5600  3300  4800  7500  7300  6200  6000  3700]


### DESCRIPTION DES FEATURES ###  
Gender : Male or Female (**Initial Data from individual**)  
Age : Age of the induvidual (**Initial Data from individual**)  
Sleep Duration : in hours decimal (**Watch**)  
Quality of Sleep : Assessed on a 0-10 scale (**assessed by questionnaire (or watch**))  
irritation level : Assessed on a 0-10 scale (**assessed by questionnaire**)  
Control level : Assessed on a 0-10 scale (**assessed by questionnaire**)  
Emotion Control level : Assessed on a 0-10 scale (**assessed by questionnaire**)  
Physical Activity Level : number of minutes spent exercising during the day (**watch or questionnaire**)  
Heart Rate : BPM nominale (**Initial Data from individual (or rate mesured during the night by watch)**)  
Heart mean : mean of BPM during the day (**watch**)  
Daily Steps : number of steps during the day (**watch**)  
Sleep Disorder : [Regular, Sleep Apnea, Insomnia'] (**Initial Data from individual**)   
Stress Level : Target feature (1 - not stressed, 2 - regular, 3 - stressed)    

In [6]:
df_health['Irritation Level'] =(( df_health['Stress Level'] + (10 - df_health['Quality of Sleep']))  // 2 + np.random.choice([-2, -1, 0, 1, 2], len(df_health)))

In [7]:
df_health['Control'] =( (10 - df_health['Stress Level'])  + np.random.choice([-2, -1, 0, 1, 2], len(df_health)))

In [8]:
df_health['Emotion Control Level'] =(( (df_health['Physical Activity Level'] / 9)  + (10 - df_health['Stress Level']))  // 2 + np.random.choice([-2, -1, 0, 1, 2], len(df_health)))

In [9]:

df_health['Mean Heart Rate'] = (
    df_health['Heart Rate'] +
    np.maximum(5, ((df_health['Stress Level'] - 5) + ((df_health['Physical Activity Level']) * 5 / df_health['Physical Activity Level'].max()))) +
    np.random.choice([-2, -1, 0, 1, 2], len(df_health))
)

In [10]:
df_health['Gender'] = df_health['Gender'].map({'Male': 0, 'Female': 1})
df_health['Sleep Disorder'] = df_health['Sleep Disorder'].map({'Sleep Apnea' : 1, 'Insomnia': 2})
df_health.fillna(0, inplace=True)

In [20]:
df_health.iloc[100:120]

#df_health.drop('Person ID', axis = 1)

,Gender,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,Heart Rate,Daily Steps,Sleep Disorder,Irritation Level,Control,Emotion Control Level,Mean Heart Rate
100,1,36,7.2,8,60,4,68,7000,0.0,2,8,4.0,71.0
101,1,36,7.2,8,60,4,68,7000,0.0,4,8,7.0,71.0
102,1,36,7.2,8,60,4,68,7000,0.0,3,8,4.0,72.0
103,0,36,6.6,5,35,7,74,4800,1.0,5,3,2.0,81.0
104,1,36,7.2,8,60,4,68,7000,1.0,4,4,8.0,71.0
105,0,36,6.6,5,35,7,74,4800,2.0,6,2,5.0,79.0
106,1,37,6.1,6,42,6,77,4200,0.0,5,3,3.0,83.0
107,0,37,7.8,8,70,4,68,7000,0.0,4,4,7.0,75.0
108,0,37,7.8,8,70,4,68,7000,0.0,4,4,4.0,75.0
109,0,37,7.4,8,60,5,68,8000,0.0,1,4,6.0,73.0


In [53]:

#df_health.to_csv('Premier_DataFrama.csv', index=False)

In [4]:
df_3features.drop(["Step_count", "Temperature"], axis = 1, inplace=True)

In [2]:
df_health.head()



,Person ID,Gender,Age,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,6.1,6,42,6,Overweight,77,4200,NaN
1,2,Male,28,6.2,6,60,8,Normal,75,10000,NaN
2,3,Male,28,6.2,6,60,8,Normal,75,10000,NaN
3,4,Male,28,5.9,4,30,8,Obese,85,3000,Sleep Apnea
4,5,Male,28,5.9,4,30,8,Obese,85,3000,Sleep Apnea


In [6]:
valeurs_sleep_disorter = df_health['Sleep Disorder'].unique()

# Affiche les valeurs uniques
print(valeurs_sleep_disorter)

[nan 'Sleep Apnea' 'Insomnia']


In [7]:
valeurs_stress_lvl = df_health['Stress Level'].unique()

print(valeurs_stress_lvl)
df_health["Stress Level"] = df_health["Stress Level"].replace({ 3 : 0, 4 : 0, 5 : 1, 6 : 1, 7 : 2, 8 : 2})


[6 8 7 4 3 5]


Santé du cœur:

Fréquence cardiaque de jour
Fréquence cardiaque de nuit
Notifications de fréquence cardiaque élevée et basse
Variations de fréquence cardiaque de nuit*
*Fonctionnalité attendue pour Q4 2023

Suivi d’activité:

Zones de fréquence cardiaque en entraînement
Minutes actives
Nombre de pas
Calories brûlées
Connexion GPS : distance, allure & dénivelé

Suivi du sommeil:

Score de qualité du sommeil
Phases de sommeil (profond, léger)
Fréquence respiratoire*
Perturbations respiratoires
Durée du sommeil
Interruptions
Régularité